In [16]:
import spacy
# from heapq import nlargest
import spacy.cli
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize #word_tokenize,
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import string
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
spacy.cli.download("en_core_web_sm")

stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
# Текст для обробки
text = """
The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.
"""

In [15]:
# Лемізація та видалення пунктуації і стоп-слів
sentences = sent_tokenize(text)
cleaned_sentences = []


def preprocess_and_lemmatize(text):
    words = nlp(text.lower())
    lemmatized_words = [token.lemma_ for token in words if token.text not in punctuation and token.text not in stop_words]
    return " ".join(lemmatized_words)

for sent in sentences:
    preprocess_lemmatized_sent = preprocess_and_lemmatize(sent)
    cleaned_sentences.append(preprocess_lemmatized_sent)

print("оброблені речення: ", cleaned_sentences)

оброблені речення:  ['\n orbiter discovery ov-103 consider eligible list national register historic place nrhp context u.s space shuttle program 1969 2011 criterion area space exploration transportation criterion c area engineering', 'achieve significance within past fifty year criterion consideration g apply', 'criterion discovery significant old three extant orbiter vehicle construct space shuttle program ssp long run american space program date third five orbiter build nasa', 'unlike mercury gemini apollo program ssp ’s emphasis cost effectiveness reusability eventually construction space station', 'include maiden voyage launch august 30 1984 discovery fly space thirty nine time four orbiter also first orbiter fly twenty mission', 'honor choose return flight vehicle challenger columbia accident', 'discovery first shuttle fly redesign srb result challenger accident first shuttle fly phase ii block ssme', 'discovery also carry hubble space telescope orbit perform two five servicing mi

In [19]:
# тренуємо модель
w2v_model = Word2Vec(sentences=cleaned_sentences, vector_size=100, window=5, min_count=1, workers=4)

# переводимо речення в вектори
sentence_vectors = []
for sent in cleaned_sentences:
    if len(sent) != 0:
        v = sum([w2v_model.wv[word] for word in sent if word in w2v_model.wv]) / (len(sent))
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

# розраховуємо матрицю подібності між векторами/реченнями
similarity_matrix = np.zeros([len(sentence_vectors), len(sentence_vectors)])
for i in range(len(sentence_vectors)):
    for j in range(len(sentence_vectors)):
        if i != j:
            similarity_matrix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

# розраховуємо важливість речень
sentence_scores = similarity_matrix.sum(axis=1)
ranked_sentences = [sentences[i] for i in np.argsort(sentence_scores)[-3:]]  # обираємо 3  найважливіших речення


In [20]:
# Формуємо summary з найбільш релевантних речень
summary = ' '.join(ranked_sentences)
print(f"{len(text)=}")
print(f"{len(summary)=}")
print(f"{summary=}")

len(text)=2908
len(summary)=928
summary='Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.'
